In [3]:
#Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#Install
!pip install catboost
!pip install xlsxwriter
!pip install catboost
!pip install scikit-optimize
!pip install xgboost
!pip install lightgbm
!pip install openpyxl

In [2]:
import os
import pandas as pd
from numpy import mean
from numpy import std
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import *
from threading import Thread
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.base import BaseEstimator, ClassifierMixin
from csv import writer

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
import csv
import time
import warnings

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from openpyxl import Workbook
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.exceptions import ConvergenceWarning


In [3]:
def load_spectrum_file(dir_path, max_subfolders=None, max_files_per_subdir=None, max_rows=None):
    """Carrega arquivos de espectro e associa rótulos numéricos às subpastas."""
    data, labels = [], []
    subfolder_labels = {}
    label_encoder = LabelEncoder()

    # Contadores para controlar a quantidade de subpastas e arquivos lidos
    subfolder_count = 0

    for root, dirs, files in os.walk(dir_path):
        # Limitar a quantidade de subpastas
        if max_subfolders is not None and subfolder_count >= max_subfolders:
            break

        for dir_name in dirs:
            subfolder_path = os.path.join(root, dir_name)
            subfolder_labels[subfolder_path] = dir_name
            subfolder_count += 1

            # Limitar a quantidade de subpastas
            if max_subfolders is not None and subfolder_count > max_subfolders:
                break

            file_count = 0
            for filename in os.listdir(subfolder_path):
                if filename.endswith('.csv'):
                    if max_files_per_subdir is not None and file_count >= max_files_per_subdir:
                        break
                    file_path = os.path.join(subfolder_path, filename)
                    try:
                        df = pd.read_csv(file_path, delimiter=',', nrows=max_rows)
                        y = df.iloc[:, 1].values  # Pega os valores da segunda coluna

                        # Adiciona o rótulo da subpasta à lista de labels
                        labels.append(subfolder_labels[subfolder_path])
                        data.append(y)
                        file_count += 1
                    except Exception as e:
                        print(f"Error reading file {file_path}: {e}")

    # Transforma os rótulos das subpastas em números
    labels = label_encoder.fit_transform(labels)
    return np.array(data), np.array(labels), label_encoder.classes_

# Exemplo de uso
print("Loading data...")
data, labels, class_names = load_spectrum_file(
    'Data_csv/',
    max_subfolders=None,
    max_files_per_subdir=None,
    max_rows=None
)
print("                                   ")
print("-----------------------------------")
print("                                   ")
print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Class names: {class_names}")
print("                                   ")
print("-----------------------------------")
print("                                   ")

X_=data
y=labels
X_ = np.nan_to_num(X_)  # Substitui NaN e inf por zero


Loading data...
                                   
-----------------------------------
                                   
Data shape: (5292, 1570)
Labels shape: (5292,)
Class names: ['Actinolite' 'Adamite' 'Aegirine' 'Aenigmatite' 'Akermanite' 'Albite'
 'Almandine' 'Amblygonite' 'Amesite' 'Analcime' 'Anatase' 'Andalusite'
 'Andradite' 'Anglesite' 'Anhydrite' 'Ankerite' 'Annite' 'Anorthite'
 'Aragonite' 'Arfvedsonite' 'Arsenopyrite' 'Astrophyllite' 'Atacamite'
 'Augelite' 'Augite' 'AxiniteFe' 'Azurite' 'Baddeleyite' 'Barrerite'
 'Barysilite' 'Baryte' 'Bertrandite' 'Beryl' 'Beudantite' 'Bikitaite'
 'Bobdownsite' 'Braunite' 'Brazilianite' 'Brookite' 'Brucite' 'Bustamite'
 'Calcite' 'Cassiterite' 'Catapleiite' 'Celestine' 'Cerussite'
 'ChabaziteCa' 'Chloritoid' 'Chondrodite' 'Chrysoberyl' 'Clinochlore'
 'Clinohumite' 'ClinoptiloliteCa' 'Clinozoisite' 'ColumbiteFe'
 'Conichalcite' 'Cordierite' 'Corundum' 'Covellite' 'Creedite' 'Crocoite'
 'Cuprite' 'Danburite' 'Datolite' 'Diamond' 'Diaspo

In [12]:
# Definição do experimento
def experiment(model_name, model, params, X_train, X_test, y_train, y_test, repetition):
    # Caminho da pasta para salvar o arquivo
    data_path = 'Results/Clássicos/SIEPE/'

    # Verifica se o diretório existe, se não, cria o diretório
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    # Nome do arquivo para salvar os resultados
    results_file = os.path.join(data_path, model_name + '.xlsx')

    # Verifica se o arquivo existe, se não, cria um arquivo vazio
    if not os.path.isfile(results_file):
        # Cria um arquivo Excel vazio
        wb = Workbook()
        wb.save(results_file)

    results = []

    # define search
    search = BayesSearchCV(model, params, scoring='accuracy', n_iter=10, refit=True, random_state=1, n_jobs=-1)

    # execute search
    result = search.fit(X_train, y_train)

    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)

    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    prec = precision_score(y_test, yhat, average='weighted', zero_division=1)
    rec = recall_score(y_test, yhat, average='weighted')
    f1 = f1_score(y_test, yhat, average='weighted')
    mcc = matthews_corrcoef(y_test, yhat)

    # store the result
    results.append([model_name, acc, rec, prec, f1, result.best_score_, result.best_params_])

    # report progress
    print(f"Repetition {repetition}: {model_name} > acc={acc:.3f}, est={result.best_score_:.3f}, cfg={result.best_params_}")

    # save results to Excel, each repetition in a new sheet
    df = pd.DataFrame(results, columns=['model', 'acc', 'rec', 'prec', 'f1', 'best_score', 'best_params'])

    # Usar o ExcelWriter com o motor 'openpyxl' para adicionar novas folhas
    with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        sheet_name = f'Repetition_{repetition}'
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [4]:
# Definição do experimento
def experiment(model_name, model, params, X_train, X_test, y_train, y_test, repetition):
    # Caminho da pasta para salvar o arquivo (cria uma pasta para cada modelo)
    model_dir = os.path.join('Results/Clássicos/SIEPE', model_name)
    
    # Verifica se o diretório existe, se não, cria o diretório
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Nome do arquivo para salvar os resultados
    results_file = os.path.join(model_dir, f'{model_name}_repetition_{repetition}.csv')
    
    # define search
    search = BayesSearchCV(model, params, scoring='accuracy', n_iter=10, refit=True, random_state=1, n_jobs=-1)
    
    # execute search
    result = search.fit(X_train, y_train)
    
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    prec = precision_score(y_test, yhat, average='weighted', zero_division=1)
    rec = recall_score(y_test, yhat, average='weighted')
    f1 = f1_score(y_test, yhat, average='weighted')
    mcc = matthews_corrcoef(y_test, yhat)
    
    # store the result
    results = [[model_name, acc, rec, prec, f1, result.best_score_, result.best_params_]]
    
    # report progress
    print(f"Repetition {repetition}: {model_name} > acc={acc:.3f}, est={result.best_score_:.3f}, cfg={result.best_params_}")
    
    # save results to CSV, each repetition in a new file
    df = pd.DataFrame(results, columns=['model', 'acc', 'rec', 'prec', 'f1', 'best_score', 'best_params'])
    df.to_csv(results_file, index=False)

In [5]:
# Definição dos modelos e parâmetros
model_params = {
    'lr': {'model': LogisticRegression(max_iter=5000),  # Aumentei o max_iter
           'params': {
               'C': Real(1e-4, 1e4, prior='log-uniform'),
               'fit_intercept': Categorical([True, False]),
               'solver': Categorical(['newton-cg', 'liblinear', 'saga'])}},

    'knn': {'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': Integer(1, 50),
                'weights': Categorical(['uniform', 'distance']),
                'algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
                'p': Integer(1, 5)}},

    'nb': {'model': GaussianNB(),
           'params': {
               'var_smoothing': Real(1e-10, 1e-1, prior='log-uniform')}},

    'dt': {'model': DecisionTreeClassifier(),
           'params': {
               'criterion': Categorical(['gini', 'entropy']),
               'splitter': Categorical(['best', 'random']),
               'max_depth': Integer(3, 30),
               'min_samples_split': Integer(2, 10),
               'min_samples_leaf': Integer(1, 10),
               'max_features': Real(0.1, 1.0, prior='uniform')}},
    
    'svm': {'model': LinearSVC(max_iter=5000), # Aumentei o max_iter
            'params': {
                'C': Real(1e-6, 1e+6, prior='log-uniform'),
                'loss': Categorical(['hinge', 'squared_hinge']),
                'tol': Real(1e-6, 1e-2, prior='log-uniform'),
                'dual': Categorical([True, False])}},

    'gpc': {'model': GaussianProcessClassifier(),
            'params': {
                'optimizer': Categorical(['fmin_l_bfgs_b', None]),
                'n_restarts_optimizer': Integer(0, 10),
                'max_iter_predict': Integer(100, 1000)}},

    'mlp': {'model': MLPClassifier(max_iter=2000),  # Aumentado max_iter
            'params': {
                'hidden_layer_sizes': Integer(10, 100),
                'activation': Categorical(['identity', 'logistic', 'tanh', 'relu']),
                'solver': Categorical(['sgd', 'adam']),
                'alpha': Real(1e-5, 1e-1, prior='log-uniform'),
                'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
                'learning_rate_init': Real(1e-4, 1e-1, prior='log-uniform')}},

    'ridge': {'model': RidgeClassifier(),
              'params': {
                  'alpha': Real(1e-4, 1e4, prior='log-uniform'),
                  'fit_intercept': Categorical([True, False]),
                  'solver': Categorical(['auto', 'cholesky', 'lsqr', 'sparse_cg', 'saga'])}},  # Removido 'sag'

    'rf': {'model': RandomForestClassifier(n_jobs=-1),  # Usar todos os núcleos disponíveis
           'params': {
               'n_estimators': Integer(10, 500),
               'criterion': Categorical(['gini', 'entropy']),
               'max_depth': Integer(3, 30),
               'min_samples_split': Integer(2, 10),
               'min_samples_leaf': Integer(1, 10),
               'max_features': Real(0.1, 1.0, prior='uniform'),
               'bootstrap': Categorical([True, False]),
               'class_weight': Categorical(['balanced', 'balanced_subsample', None])}},

    'qda': {'model': QuadraticDiscriminantAnalysis(),
            'params': {
                'reg_param': Real(0, 1, prior='uniform'),
                'store_covariance': Categorical([True, False]),
                'tol': Real(1e-5, 1e-1, prior='log-uniform')}},

    'ada': {'model': AdaBoostClassifier(),
            'params': {
                'n_estimators': Integer(10, 500),
                'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                'algorithm': Categorical(['SAMME'])}},  # Alterado para apenas 'SAMME'

    'gbc': {'model': GradientBoostingClassifier(),
            'params': {
                'n_estimators': Integer(10, 500),
                'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                'max_depth': Integer(3, 10),
                'min_samples_split': Integer(2, 10),
                'min_samples_leaf': Integer(1, 10),
                'max_features': Real(0.1, 1.0, prior='uniform'),
                'subsample': Real(0.1, 1.0, prior='uniform')}},

    'lda': {'model': LinearDiscriminantAnalysis(),
            'params': {
                'solver': Categorical(['lsqr', 'eigen']),
                'shrinkage': Real(0, 1, prior='uniform'),
                'tol': Real(1e-6, 1e-4, prior='log-uniform')}},

    'et': {'model': ExtraTreesClassifier(n_jobs=-1),  # Usar todos os núcleos disponíveis
           'params': {
               'n_estimators': Integer(10, 500),
               'criterion': Categorical(['gini', 'entropy']),
               'max_depth': Integer(3, 30),
               'min_samples_split': Integer(2, 10),
               'min_samples_leaf': Integer(1, 10),
               'max_features': Real(0.1, 1.0, prior='uniform'),
               'bootstrap': Categorical([True, False]),
               'class_weight': Categorical(['balanced', 'balanced_subsample', None])}},

    'xgboost': {'model': XGBClassifier(),
                'params': {
                    'learning_rate': Real(0.01, 0.3, prior='uniform'),
                    'n_estimators': Integer(50, 500),
                    'max_depth': Integer(3, 10),
                    'min_child_weight': Integer(1, 10),
                    'gamma': Real(0, 1, prior='uniform'),
                    'subsample': Real(0.5, 1, prior='uniform'),
                    'colsample_bytree': Real(0.5, 1, prior='uniform'),
                    'reg_alpha': Real(0, 1, prior='uniform'),
                    'reg_lambda': Real(1, 3, prior='uniform'),
#                    'scale_pos_weight': Real(1, 5, prior='uniform')
                }},

    'lightgbm': {'model': LGBMClassifier(verbose=-1),
                 'params': {
                     'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                     'n_estimators': Integer(10, 500),
                     'num_leaves': Integer(2, 100),
                     'max_depth': Integer(3, 10),
                     'min_child_samples': Integer(1, 50),
                     'min_child_weight': Real(1e-5, 1e-3, prior='log-uniform'),
                     'subsample': Real(0.1, 1.0, prior='uniform'),
                     'colsample_bytree': Real(0.1, 1.0, prior='uniform'),
                     'reg_alpha': Real(0, 1, prior='uniform'),
                     'reg_lambda': Real(0, 1, prior='uniform')}},

    'catboost': {'model': CatBoostClassifier(verbose=0),
                 'params': {
                     'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                     'iterations': Integer(10, 500),
                     'depth': Integer(3, 10),
                     'l2_leaf_reg': Real(1, 10, prior='uniform'),
                     'border_count': Integer(1, 255),
                     'bagging_temperature': Real(0, 1, prior='uniform'),
                     'random_strength': Real(1e-9, 10, prior='log-uniform')}}
}


In [ ]:
# Loop para realizar 30 iterações de train_test_split e executar experimentos
# Supressão de avisos de convergência
warnings.filterwarnings("ignore", category=ConvergenceWarning)
for i in range(30):
    # Realiza o particionamento dos dados
    X_train, X_test, y_train, y_test = train_test_split(X_, y, train_size=0.7, test_size=0.3, random_state=1, shuffle=True, stratify=y)

    # Lista para armazenar a referência das threads
    threads = []

    # Cria o pool de threads
    pool = ThreadPoolExecutor(max_workers=8)

    for model_name, mp in model_params.items():
        # Adiciona o experimento na lista de threads
        exp = pool.submit(experiment, model_name, mp['model'], mp['params'], X_train, X_test, y_train, y_test, i + 1)

        # Armazena a referência da thread
        threads.append(exp)
        time.sleep(0.1)

    # Aguarda a finalização das threads
    for exp in as_completed(threads):
        exp.result()

    # Fecha o pool
    pool.shutdown()

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Repetition 1: nb > acc=0.899, est=0.886, cfg=OrderedDict([('var_smoothing', 0.0029757134630289256)])


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which 

Repetition 1: dt > acc=0.738, est=0.705, cfg=OrderedDict([('criterion', 'entropy'), ('max_depth', 15), ('max_features', 0.6380599656985619), ('min_samples_leaf', 3), ('min_samples_split', 7), ('splitter', 'best')])


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means 

Repetition 1: knn > acc=0.933, est=0.921, cfg=OrderedDict([('algorithm', 'brute'), ('n_neighbors', 2), ('p', 4), ('weights', 'uniform')])


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which 